# P실무 (자율주행)

## 조건
- 지정된 트랙을 따라 카메라 탑재 자동차가 주행
- 장애물 감지 및 대응
    - 정지 인식: 정지 후 같은 음으로 버저 3회
    - 위험 인식: 정지 후 다른 음으로 버저 3회
    - 'O' 표식 인식 시 해당 위치에 주차
- 주행 경로: 검정색
- 바운더리: 흰색

## 주요 Cascade 학습 환경설정 값

- **Sample Size:** 24 x 24  
  - 특징 검출용 윈도우 크기
- **Feature Type:** HAAR (BASIC)  
  - 사용되는 특징 유형
- **Number of Stages:** 20  
  - 분류기 강화를 위한 단계 수
- **Threads:** 5  
  - 병렬처리 시 사용할 스레드 수
- **Minimal Hit Rate:** 0.995  
  - 검출기 정확도 최소 기준 (검출률)
- **Maximal False Alarm Rate:** 0.5  
  - 허용 가능한 최대 오검출율
- **Boost Type:** GAB  
  - 부스팅 알고리즘 타입
- **Maximal Weak Trees:** 100  
  - 각 스테이지당 약한 분류기(weak learner) 최대 개수

## 코드 구성 요소 설명
- **Manager 클래스**
    - 시스템 전체 제어(초기화, 메인 루프)
    - 카메라, 자동차 제어, 이미지 처리, 버저, Cascade 로더 초기화
    - 실시간 프레임 캡처 후 주행방향 결정
    - X, O_left, O_right, O_total, threat, stop 등 기호 인식 시 해당 동작 수행(정지, 회전 등)

- **Camera 클래스**
    - OpenCV 기반 카메라 초기화 및 설정(해상도, 밝기, 대비 등)
    - `capture_frame()` 통해 실시간 영상 캡처
    - 캡처한 프레임을 이미지 처리기로 전달

- **CarControl 클래스**
    - 자동차 구동 제어
    - 방향 명령(UP, LEFT, RIGHT, REDUCED_LEFT, REDUCED_RIGHT) 별 모터 속도 조정
    - `stop()` 메서드로 긴급 정지 가능

- **ImageProcessor 클래스**
    - 영상 전처리(그레이스케일, 투시 변환 등)
    - 히스토그램 분석 통한 주행 방향 결정
    - Cascade Classifier 활용 기호 인식

- **Buzzer 클래스**
    - GPIO 활용 버저 제어
    - 이벤트별 버저 음색 및 패턴(정지, 위협 시각적 알림)

- **CascadeLoader 클래스**
    - OpenCV Cascade 분류기 로드 담당
    - 특정 기호 인식용 모델 불러오기

## 동작 흐름

```text
┌───────────────┐        ┌─────────────────┐
│    Camera     │        │   CascadeLoader │
│   영상 캡처   │        │  기호 인식 모델  │
└─────┬─────────┘        └───────┬─────────┘
      │                          │
      ▼                          ▼
┌───────────────┐        ┌────────────────┐
│ ImageProcessor│  <---- │     Manager    │
│  전처리 & 분석 │        │    메인 제어    │
└─────┬─────────┘        └───────┬────────┘
      │                          │
      ▼                          ▼
┌───────────────┐        ┌───────────────┐
│   CarControl  │        │     Buzzer    │
│   차량 제어    │        │   음향 알림   │
└───────────────┘        └───────────────┘

### 처리 단계
1. 카메라로부터 실시간 프레임 수신
2. 프레임 전처리(그레이스케일 변환, 투시변환) 및 히스토그램 분석
3. Cascade 분류기로 특정 기호(X, O, threat, stop 등) 탐지
4. 기호 종류 및 주행 방향 분석 결과에 따라 자동차 제어(CarControl) 명령
5. 상황에 따라 버저(Buzzer) 알림
6. 사용자가 키 입력 시 정지 또는 종료
7. 예외 발생 시 안전 정지 후 리소스 해제 및 종료

### 메인 루프 알고리즘
```text

┌──────────────────┐
│ camera.capture_  │
│    _frame()      │
└───┬──────────────┘
    │ frame
    ▼
┌────────────────┐
│ image_processor│
│.process_frame()│
└─────┬──────────┘
      │ processed_frame
      ▼
┌───────────────────┐
│ image_processor   │
│ .analyze()        │ → 주행 방향(direction)
└─────┬─────────────┘
      │ direction
      ▼
┌───────────────────┐
│ image_processor   │
│ .detect_symbol()  │ → 기호(symbol)
└─────┬─────────────┘
      │ symbol
      ▼
┌──────────────────────────────────────────────────────────┐
│                        판단                              │
│                                                          │
│ if symbol == 'stop'   : 차 정지 + 버저(같은 음 3회)       │
│ if symbol == 'threat' : 차 정지 + 버저(다른 음 3회)       │
│ if symbol == 'O'      : 주차(차 정지)                     │
│ else                  : direction대로 차량 구동           │
└──────────────────────────────────────────────────────────┘
